In [1]:
# DATA format
'''

# Retrieve Data
# Data format. Each item in the list represents userid, video_id, num of times viewed / rating
user_ad_views = [[1,1,5],
    [3,2,1],
                 [3,1,1],
                 [1,2,10],
                 [2,4,1],
       [2,1,6],
       [2,2,2],
       [3,3,1]]
user_ad_views = sc.parallelize(user_ad_views)
user_ad_views = user_ad_views.map(lambda x:(x[0], x[1], x[2]))

ad_names = [(1,"one"),(2,"two"),(3,"three"),(4,"four")]
ad_names = sc.parallelize(ad_names)
# print (user_ad_views.collect())

'''

[(1, 1, 5), (3, 2, 1), (3, 1, 1), (1, 2, 10), (2, 4, 1), (2, 1, 6), (2, 2, 2), (3, 3, 1)]


In [30]:
# MongoDB connection
import pymongo

client = pymongo.MongoClient('mongodb://152.46.20.123:27017')
db = client.testgridfs

# USERS data
users = db.users
data = []
for obj in users.find():
    if(len(obj['videosViewed']) >= 1):
        for viewed in obj['videosViewed']:
            data.append((obj['userId'], viewed['videoId'], 1))
print (data)

# VIDEOS information
videos = db.videos
ad_names = []
for obj in videos.find():
    ad_names.append((obj['videoId'], obj['title']))
print (ad_names)


[(2, 1, 1), (2, 6, 1), (2, 7, 1), (2, 2, 1), (2, 3, 1), (2, 4, 1), (2, 5, 1), (2, 6, 1), (2, 7, 1), (2, 8, 1), (2, 9, 1), (2, 10, 1), (3, 1, 1), (3, 7, 1), (4, 1, 1), (4, 7, 1), (4, 4, 1), (4, 5, 1), (4, 10, 1), (5, 6, 1), (5, 7, 1), (5, 10, 1), (6, 1, 1), (6, 7, 1), (6, 2, 1), (6, 4, 1), (6, 8, 1), (7, 1, 1), (7, 6, 1), (7, 5, 1), (7, 6, 1), (7, 7, 1), (7, 8, 1), (7, 10, 1), (8, 1, 1), (8, 10, 1), (9, 5, 1), (9, 6, 1), (9, 7, 1), (9, 8, 1), (10, 1, 1), (10, 6, 1), (10, 7, 1), (10, 9, 1)]
[(1, u'laptop1'), (2, u'laptop2'), (3, u'clothing1'), (4, u'clothing2'), (5, u'med1'), (6, u'grocery1'), (7, u'sport1'), (8, u'sport2'), (9, u'footwear1'), (10, u'footwear2'), (11, u'laptop4'), (12, u'laptop3'), (13, u'cothing3'), (14, u'clothing4'), (15, u'cos1'), (16, u'cos3'), (17, u'movie1'), (18, u'movie2'), (19, u'rest1'), (20, u'rest2'), (21, u'groc1'), (22, u'groc2'), (23, u'sport3'), (24, u'mobile5')]


In [31]:
# Forming user-ads-views RDDs
user_ad_views = sc.parallelize(data)
user_ad_views = user_ad_views.map(lambda x:((x[0], x[1]), x[2]))
user_ad_views = user_ad_views.reduceByKey(lambda x, y: x+y)
user_ad_views = user_ad_views.map(lambda x:(x[0][0], x[0][1], x[1]))
print (user_ad_views.take(5))

# Forming ad-names RDDS
ad_names = sc.parallelize(ad_names)
print (ad_names.take(2))

[(6, 4, 1), (6, 8, 1), (4, 10, 1), (2, 8, 1), (7, 7, 1)]
[(1, u'laptop1'), (2, u'laptop2')]


In [32]:
# Data split for cross validation
trainData, validationData, testData = user_ad_views.randomSplit([40, 40, 20], 13)
print (trainData.collect())

[(6, 8, 1), (4, 10, 1), (9, 5, 1), (3, 7, 1), (2, 4, 1), (5, 10, 1), (7, 8, 1), (2, 3, 1), (9, 6, 1), (4, 1, 1), (2, 6, 2), (3, 1, 1), (5, 7, 1), (10, 6, 1), (4, 4, 1), (6, 2, 1), (4, 7, 1), (2, 9, 1), (6, 1, 1), (2, 1, 1), (10, 1, 1), (10, 9, 1)]


In [33]:
unique_user = user_ad_views.map(lambda x:(x[0],(x[1], x[2]))).groupByKey().map(lambda x:x[0])
unique_ad = ad_names.map(lambda x:x[0])
all_pairs = unique_user.cartesian(unique_ad)
trainData_m = trainData.map(lambda x:(x[0], x[1]))
print (unique_user.take(5))
print (unique_ad.take(5))
print (all_pairs.take(5))
print (trainData_m.take(5))

[8, 4, 9, 5, 2]
[1, 2, 3, 4, 5]
[(8, 1), (8, 2), (8, 3), (8, 4), (8, 5)]
[(6, 8), (4, 10), (9, 5), (3, 7), (2, 4)]


In [34]:
# Evaluating a model

def computeAvg(rdd):
    userid, pre, to_c = rdd[0], list(rdd[1][0]), list(rdd[1][1])
    to_c = sc.parallelize(to_c).map(lambda x:x[0]).collect()
    X = len(to_c)
    pre = sc.parallelize(pre).sortBy(lambda x:-x[1]).map(lambda x:x[0]).collect()[:X]
    same = 0
    for i in pre:
        if i in to_c: same+=1
    return same*1.0/X
        
    
def modelEval(model, validationData):
    to_be_predicted = all_pairs.subtract(trainData_m)
    predicted_data = model.predictAll(to_be_predicted).map(lambda x:(x[0], (x[1], x[2]))).groupByKey()
    to_compare = validationData.subtract(trainData).map(lambda x:(x[0], (x[1], x[2]))).groupByKey()
    merge = predicted_data.join(to_compare)
    total = 0
    for each in merge.collect():
        total+=computeAvg(each)
    return (total*1.0/50)

In [35]:
# Building the model
from pyspark.mllib.recommendation import *
import random
from operator import *

for rank in [1, 2, 3]:
    model = ALS.trainImplicit(trainData, rank = rank, seed = 345)
    modelScore = modelEval(model, validationData)
    print "The model score for rank %d is %f"%(rank, modelScore)

The model score for rank 1 is 0.036667
The model score for rank 2 is 0.026667
The model score for rank 3 is 0.030000


In [36]:
bestModel = ALS.trainImplicit(trainData, rank=1, seed=345)
ans = modelEval(bestModel, testData)
print "Model Score for best model, rank = 10 is %f"%(ans)


Model Score for best model, rank = 10 is 0.010000


In [37]:
# Recommend now
top = bestModel.recommendProducts(2, 2)
print (top)

[Rating(user=2, product=1, rating=1.1812443516124844), Rating(user=2, product=6, rating=0.6369860890532593)]


In [43]:
# Map recommendations to ad names 
recommendations = bestModel.recommendProductsForUsers(10)
print (recomm)

[(2, (Rating(user=2, product=1, rating=1.1812443516124844), Rating(user=2, product=6, rating=0.6369860890532593), Rating(user=2, product=4, rating=0.6095432391537656), Rating(user=2, product=7, rating=0.575439891282695), Rating(user=2, product=9, rating=0.5656834342127723), Rating(user=2, product=10, rating=0.39090900155405706), Rating(user=2, product=3, rating=0.32696971434747546), Rating(user=2, product=8, rating=0.17352926853684636), Rating(user=2, product=2, rating=0.15536263477089474), Rating(user=2, product=5, rating=0.07122940141314693))), (3, (Rating(user=3, product=1, rating=0.6478738501945855), Rating(user=3, product=6, rating=0.34936601345181373), Rating(user=3, product=4, rating=0.3343145088241499), Rating(user=3, product=7, rating=0.3156099391391436), Rating(user=3, product=9, rating=0.3102588418852008), Rating(user=3, product=10, rating=0.21440078808997498), Rating(user=3, product=3, rating=0.17933218257691763), Rating(user=3, product=8, rating=0.09517512204392631), Ratin